# Nifti Read Example

The purpose of this notebook is to illustrate reading Nifti files and iterating over patches of the volumes loaded from them.

In [1]:
%matplotlib inline

import os
import sys
from glob import glob
import tempfile

import numpy as np
import matplotlib.pyplot as plt
import nibabel as nib


import torch
from torch.utils.data import DataLoader
import torchvision.transforms as transforms

sys.path.append('..') # assumes this is where MONAI is

from monai import application, data, networks, utils
from monai.data.readers import NiftiDataset
from monai.data.transforms import AddChannel, Transpose, Rescale, ToTensor, UniformRandomPatch, GridPatchDataset

application.config.print_config()

MONAI version: 0.0.1
Python version: 3.7.3 (default, Mar 27 2019, 22:11:17)  [GCC 7.3.0]
Numpy version: 1.16.4
Pytorch version: 1.3.1
Ignite version: 0.2.1


Define a function for creating test images and segmentations:

In [2]:
def create_test_image_3d(height, width, depth, numObjs=12, radMax=30, noiseMax=0.0, numSegClasses=5):
    '''Return a noisy 3D image and segmentation.'''
    image = np.zeros((width, height,depth))

    for i in range(numObjs):
        x = np.random.randint(radMax, width - radMax)
        y = np.random.randint(radMax, height - radMax)
        z = np.random.randint(radMax, depth - radMax)
        rad = np.random.randint(5, radMax)
        spy, spx, spz = np.ogrid[-x:width - x, -y:height - y, -z:depth - z]
        circle = (spx * spx + spy * spy + spz * spz) <= rad * rad

        if numSegClasses > 1:
            image[circle] = np.ceil(np.random.random() * numSegClasses)
        else:
            image[circle] = np.random.random() * 0.5 + 0.5

    labels = np.ceil(image).astype(np.int32)

    norm = np.random.uniform(0, numSegClasses * noiseMax, size=image.shape)
    noisyimage = utils.arrayutils.rescale_array(np.maximum(image, norm))

    return noisyimage, labels

Create a number of test Nifti files:

In [3]:
tempdir = tempfile.mkdtemp()

for i in range(5):
    im, seg = create_test_image_3d(256,256,256)
    
    n = nib.Nifti1Image(im, np.eye(4))
    nib.save(n, os.path.join(tempdir, 'im%i.nii.gz'%i))
    
    n = nib.Nifti1Image(seg, np.eye(4))
    nib.save(n, os.path.join(tempdir, 'seg%i.nii.gz'%i))

Create a data loader which yields uniform random patches from loaded Nifti files:

In [4]:
images = sorted(glob(os.path.join(tempdir,'im*.nii.gz')))
segs = sorted(glob(os.path.join(tempdir,'seg*.nii.gz')))

imtrans=transforms.Compose([
    Rescale(),
    AddChannel(),
    UniformRandomPatch((64, 64, 64)),
    ToTensor()
])    

segtrans=transforms.Compose([
    AddChannel(),
    UniformRandomPatch((64, 64, 64)),
    ToTensor()
])    
    
ds = NiftiDataset(images, segs, imtrans, segtrans)

loader = DataLoader(ds, batch_size=10, num_workers=2, pin_memory=torch.cuda.is_available())
im, seg = utils.mathutils.first(loader)
print(im.shape, seg.shape)

torch.Size([5, 1, 64, 64, 64]) torch.Size([5, 1, 64, 64, 64])


Alternatively create a data loader which yields patches in regular grid order from loaded images:

In [5]:
imtrans=transforms.Compose([
    Rescale(),
    AddChannel(),
    ToTensor()
])    

segtrans=transforms.Compose([
    AddChannel(),
    ToTensor()
])    
    
ds = NiftiDataset(images, segs, imtrans, segtrans)
ds = GridPatchDataset(ds, (64, 64, 64))

loader = DataLoader(ds, batch_size=10, num_workers=2, pin_memory=torch.cuda.is_available())
im, seg = utils.mathutils.first(loader)
print(im.shape, seg.shape)

torch.Size([10, 1, 64, 64, 64]) torch.Size([10, 1, 64, 64, 64])


In [6]:
!rm -rf {tempdir}